In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

# Import the API key.
from config import weather_api_key

# Import the requests library.
import requests

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=100)
lngs = np.random.uniform(low=-180.000, high=180.000, size=100)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

74

In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [6]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    print(city_url)
    
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_data_all=requests.get(city_url)
        city_weather = city_data_all.json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city.
    except :
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=c7ff403200ab6145d3262ed8048eedfb&q=sompeta
Processing Record 1 of Set 1 | sompeta
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=c7ff403200ab6145d3262ed8048eedfb&q=cidreira
Processing Record 2 of Set 1 | cidreira
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=c7ff403200ab6145d3262ed8048eedfb&q=carnarvon
Processing Record 3 of Set 1 | carnarvon
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=c7ff403200ab6145d3262ed8048eedfb&q=upernavik
Processing Record 4 of Set 1 | upernavik
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=c7ff403200ab6145d3262ed8048eedfb&q=norman+wells
Processing Record 5 of Set 1 | norman wells
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=c7ff403200ab6145d3262ed8048eedfb&q=arraial+do+cabo
Processing Record 6 of Set 1 | arraial do cabo
http

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=c7ff403200ab6145d3262ed8048eedfb&q=bergen
Processing Record 4 of Set 2 | bergen
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=c7ff403200ab6145d3262ed8048eedfb&q=fairbanks
Processing Record 5 of Set 2 | fairbanks
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=c7ff403200ab6145d3262ed8048eedfb&q=zhigansk
Processing Record 6 of Set 2 | zhigansk
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=c7ff403200ab6145d3262ed8048eedfb&q=acapulco
Processing Record 7 of Set 2 | acapulco
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=c7ff403200ab6145d3262ed8048eedfb&q=yining
Processing Record 8 of Set 2 | yining
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=c7ff403200ab6145d3262ed8048eedfb&q=barentsburg
Processing Record 9 of Set 2 | barentsburg
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?units=Imperia

In [8]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Sompeta,18.9333,84.6000,80.42,84,79,7.70,IN,2022-05-08 21:11:14
1,Cidreira,-30.1811,-50.2056,66.47,78,0,13.87,BR,2022-05-08 21:11:14
2,Carnarvon,-24.8667,113.6333,60.87,94,0,8.05,AU,2022-05-08 21:11:15
3,Upernavik,72.7868,-56.1549,19.51,80,73,6.53,GL,2022-05-08 21:01:45
4,Norman Wells,65.2820,-126.8329,43.29,56,100,1.59,CA,2022-05-08 21:07:47
5,Arraial Do Cabo,-22.9661,-42.0278,75.06,64,75,4.61,BR,2022-05-08 21:07:15
6,Tomatlan,19.9333,-105.2500,87.73,47,6,10.07,MX,2022-05-08 21:11:16
7,Tuktoyaktuk,69.4541,-133.0374,35.17,88,43,14.63,CA,2022-05-08 21:08:49
8,Cherskiy,68.7500,161.3000,22.89,92,100,11.63,RU,2022-05-08 21:11:17
9,Urengoy,65.9667,78.3667,33.37,98,100,12.88,RU,2022-05-08 21:11:17


In [9]:
new_column_order = ["City", "Country","Date","Lat","Lng", "Max Temp","Humidity", "Cloudiness","Wind Speed"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Sompeta,IN,2022-05-08 21:11:14,18.9333,84.6000,80.42,84,79,7.70
1,Cidreira,BR,2022-05-08 21:11:14,-30.1811,-50.2056,66.47,78,0,13.87
2,Carnarvon,AU,2022-05-08 21:11:15,-24.8667,113.6333,60.87,94,0,8.05
3,Upernavik,GL,2022-05-08 21:01:45,72.7868,-56.1549,19.51,80,73,6.53
4,Norman Wells,CA,2022-05-08 21:07:47,65.2820,-126.8329,43.29,56,100,1.59
5,Arraial Do Cabo,BR,2022-05-08 21:07:15,-22.9661,-42.0278,75.06,64,75,4.61
6,Tomatlan,MX,2022-05-08 21:11:16,19.9333,-105.2500,87.73,47,6,10.07
7,Tuktoyaktuk,CA,2022-05-08 21:08:49,69.4541,-133.0374,35.17,88,43,14.63
8,Cherskiy,RU,2022-05-08 21:11:17,68.7500,161.3000,22.89,92,100,11.63
9,Urengoy,RU,2022-05-08 21:11:17,65.9667,78.3667,33.37,98,100,12.88


In [10]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")